In [ ]:
import sys
import os
sys.path.append('../')
import pathlib as pl
from SymEigen import *
import math

from sympy import symbols, Matrix, log, sqrt
from project_dir import backend_source_dir

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")
Gen.DisableLatexComment()

k = Eigen.Scalar('k')
L0 = Eigen.Scalar("L0")
radius = Eigen.Scalar("r")
_pi = Eigen.Scalar("pi")

X = Eigen.Vector("X", 9)
X0 = X[0:3,0]
X1 = X[3:6,0]
X2 = X[6:9,0]

e0 = X1 - X0
e1 = X2 - X1

In [ ]:
kappa = e0.cross(e1) * 2 / (sqrt(e0.dot(e0) * e1.dot(e1)) + e0.dot(e1))
kappa

In [ ]:
alpha = k * radius**4 * _pi / 4
alpha

In [ ]:
E = alpha * kappa.dot(kappa) / L0
E

In [ ]:
dEdX = VecDiff(E, X)
ddEddX = VecDiff(dEdX, X)

In [ ]:
Cl = Gen.Closure(k, X, L0, radius, _pi)

In [ ]:
s = f'''
{Cl("E",E)}
{Cl("dEdX",dEdX)}
{Cl("ddEddX",ddEddX)}
'''
print(s)

f = open( backend_source_dir('cuda') / 'finite_element/constitutions/sym/kirchhoff_rod_bending.inl', 'w')
f.write(s)
f.close()